In [3]:
import numpy as np
import os
import pandas as pd
CLEAN_DATA_DIR = "../data/clean/"

In [22]:
unfilled  = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_UNFILLED.csv"))
train_A = unfilled[[col for col in unfilled.columns if col not in ['hhid']]]
# the train_set as a result of inner joining with SubjectivePoverty (unfilled data) => ~5300 rows

subjects_in_train_A = set(train_A['psu_hh_idcode'])
filled = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_FILLED.csv")) # all training data that doesn't appear in unfilled data (these are without labels) => ~15000 ish rows?
filled = filled[~filled['psu_hh_idcode'].isin(subjects_in_train_A)]
filled_X = filled[[col for col in filled.columns if col not in ['hhid', 'rating_filled']]]
train_B = filled_X

print("train_A: ", train_A.shape)
display(train_A.head())
print("train_B: ", train_B.shape)
display(train_B.head())

train_A:  (5334, 14)


,psu_hh_idcode,subjectivePoverty_rating,q02,q03,q05,q09,q23,Q01,Q03,Q06,Q07,Q08,Q11,Q19
0,30_8_1,4,1,1,44,0,0,1,1,2.0,1.0,2.0,13.0,2.0
1,194_1_2,1,2,2,48,0,0,1,1,2.0,0.0,2.0,13.0,2.0
2,224_6_1,3,1,1,61,0,0,1,1,2.0,0.0,2.0,13.0,2.0
3,323_10_1,5,1,1,66,0,0,1,1,2.0,0.0,2.0,13.0,2.0
4,428_10_1,4,2,1,72,0,0,1,1,1.0,0.0,2.0,14.0,2.0


train_B:  (14929, 13)


,psu_hh_idcode,q02,q03,q05,q09,q23,Q01,Q03,Q06,Q07,Q08,Q11,Q19
5334,1_2_2,2,2,47,0,0,1.0,1.0,2.0,1.0,2.0,13.0,2.0
5335,1_3_2,2,2,56,0,0,1.0,1.0,2.0,1.0,2.0,13.0,2.0
5336,1_3_3,1,4,27,0,2,1.0,1.0,2.0,1.0,2.0,13.0,2.0
5337,1_5_2,2,2,49,0,0,1.0,1.0,1.0,0.0,2.0,2.0,2.0
5338,1_5_3,1,4,36,0,2,1.0,1.0,2.0,0.0,2.0,2.0,2.0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# LEARNING weights with training data:
#   For each learner:
    # train the learner on train_A
    # predict probabilities for train_B

P_RF = pd.read_csv(file_path_to_p_rf)
P_XGB = pd.read_csv(file_path_to_p_xgb)
P_SVM = pd.read_csv(file_path_to_p_svm)

# aligned on id column. make sure each row corresponds to the same subject
assert all(P_RF['psu_hh_idcode'] == P_XGB['psu_hh_idcode'])
assert all(P_XGB['psu_hh_idcode'] == P_SVM['psu_hh_idcode'])

P_RF = P_RF.drop(columns=['psu_hh_idcode'])
P_XGB = P_XGB.drop(columns=['psu_hh_idcode'])
P_SVM = P_SVM.drop(columns=['psu_hh_idcode'])


# logistic regression on probabilities of each learner on train_B
X_stack = pd.concat([P_RF, P_XGB, P_SVM], axis=1)


stack_learner = LogisticRegression(multi_class='multinomial', max_iter=1000)
stack_learner.fit(X_train, y_train)

# 